# Версия 4: Мощность и параллелизм

Четвёртая версия модернизирует автоэнкодер и добавляет многопоточность. Системная энтропия (загрузка процессора) повышает случайность ключей.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import os
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import psutil
from scipy.stats import entropy

## Изменения

- **Кастомная активация**: `chaos_activation`.
- **Регуляризация**: `VarianceRegularizer`.
- **Многопоточность**: `ThreadPoolExecutor`.
- **Энтропия**: `cpu_usage`.

In [4]:
# Логистическое отображение
def logistic_map(x, r=3.99):
    return r * x * (1 - x)

# Генерация хаотичного изображения
def generate_logistic_map_image(image_size=28, initial_value=0.4, r=3.99):
    iterations = image_size * image_size
    x = initial_value
    chaotic_sequence = []
    for _ in range(iterations):
        x = logistic_map(x, r)
        chaotic_sequence.append(x)
    img = np.array(chaotic_sequence).reshape((image_size, image_size))
    return img[..., np.newaxis]

# Генерация набора хаотичных изображений
def generate_logistic_map_images_dataset(num_images, image_size=28, r=3.99):
    images = [generate_logistic_map_image(image_size, np.random.uniform(0.1, 0.9), r) for _ in range(num_images)]
    return np.array(images)

# Кастомная активация
def chaos_activation(x):
    return tf.sin(8.0 * x) + 0.5 * tf.tanh(4.0 * x)

# Регуляризация дисперсии
class VarianceRegularizer(layers.Layer):
    def __init__(self, lambda_reg=0.01, **kwargs):
        super(VarianceRegularizer, self).__init__(**kwargs)
        self.lambda_reg = lambda_reg
    def call(self, inputs):
        variance_loss = -self.lambda_reg * tf.reduce_mean(tf.math.reduce_variance(inputs, axis=0))
        self.add_loss(variance_loss)
        return inputs

# Построение автоэнкодера
def build_autoencoder(image_size=(28, 28)):
    input_img = keras.Input(shape=(*image_size, 1))
    x = layers.Flatten()(input_img)
    x = layers.Dense(128)(x)
    x = layers.Activation(chaos_activation)(x)
    latent = layers.Dense(64, name="latent")(x)
    latent = layers.Activation(chaos_activation)(latent)
    latent = VarianceRegularizer(lambda_reg=0.01)(latent)
    decoded = layers.Dense(128, activation='relu')(latent)
    decoded = layers.Dense(np.prod(image_size), activation='sigmoid')(decoded)
    decoded = layers.Reshape((*image_size, 1))(decoded)
    autoencoder = keras.Model(input_img, decoded)
    encoder = keras.Model(input_img, latent)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# Динамическое дообучение
def dynamic_retraining_with_chaos_maps(autoencoder, encoder, num_images=500, epochs=2):
    new_images = generate_logistic_map_images_dataset(num_images=num_images, image_size=28, r=3.99)
    for layer in autoencoder.layers[:-3]:
        layer.trainable = False
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='mse')
    autoencoder.fit(new_images, new_images, epochs=epochs, batch_size=32, verbose=0)
    return autoencoder, encoder

# Генерация ключей RSA с многопоточностью
def generate_enhanced_rsa_keys_from_image(encoder):
    image = generate_logistic_map_image()[np.newaxis, ...]
    latent_repr = encoder.predict(image, verbose=0)
    system_entropy = os.urandom(32)
    cpu_usage = str(psutil.cpu_percent(interval=0.01)).encode('utf-8')
    timestamp = datetime.utcnow().isoformat().encode('utf-8')
    combined = latent_repr.tobytes() + system_entropy + timestamp + cpu_usage
    kdf = PBKDF2HMAC(algorithm=hashes.SHA512(), length=64, salt=system_entropy[:16], iterations=5000, backend=default_backend())
    derived_key = kdf.derive(combined)
    seed_p, seed_q = derived_key[:32], derived_key[32:]
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=1024, backend=default_backend())
    public_key = private_key.public_key()
    return private_key, public_key, system_entropy[:16], timestamp

## Тестирование

Тесты проверяют шифрование, вариативность и дообучение, выводя:
- MSE автоэнкодера.
- Время генерации ключей.
- Эффект лавины.
- Вариативность латентного пространства.
- Эффект дообучения.
- Энтропия ключей.


In [5]:
import unittest
import time
from scipy.stats import entropy

class TestImageBasedCrypto(unittest.TestCase):
    def setUp(self):
        self.autoencoder, self.encoder = build_autoencoder()
        chaotic_images = generate_logistic_map_images_dataset(1000, image_size=28)
        chaotic_images = chaotic_images.astype('float32')
        self.autoencoder.fit(chaotic_images, chaotic_images, epochs=5, batch_size=32, verbose=0)
        self.autoencoder, self.encoder = dynamic_retraining_with_chaos_maps(self.autoencoder, self.encoder)
        self.image = chaotic_images[0:1]

    def test_encryption_decryption(self):
        start_time = time.time()
        private_key, public_key, salt, _ = generate_enhanced_rsa_keys_from_image(self.encoder)
        gen_time = time.time() - start_time
        message = b"Hello, World!"
        ciphertext = public_key.encrypt(
            message,
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA512()), algorithm=hashes.SHA512(), label=None)
        )
        plaintext = private_key.decrypt(
            ciphertext,
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA512()), algorithm=hashes.SHA512(), label=None)
        )
        mse = float(self.autoencoder.evaluate(self.image, self.image, verbose=0))
        img2 = generate_logistic_map_image(initial_value=0.41)
        private_key2, _, _, _ = generate_enhanced_rsa_keys_from_image(self.encoder)
        avalanche = np.mean(np.abs(np.array(private_key.private_numbers().p) - np.array(private_key2.private_numbers().p)))
        # Энтропия соли
        hist, _ = np.histogram(salt, bins=256, range=(0, 256), density=True)
        ent = entropy(hist, base=2)
        self.assertEqual(message, plaintext, "Decryption failed")
        print(f"Test encryption_decryption passed - MSE: {mse:.4f}, Generation Time: {gen_time:.3f}s, Avalanche Effect: {avalanche:.4f}, Entropy: {ent:.3f} bits/byte")

    def test_latent_variation(self):
        latent1 = self.encoder.predict(self.image, verbose=0)
        new_image = generate_logistic_map_image(initial_value=0.41)
        latent2 = self.encoder.predict(new_image[np.newaxis, ...], verbose=0)
        diff = np.mean(np.abs(latent1 - latent2))
        self.assertGreater(diff, 0.001, "Latent variation too low")
        print(f"Test latent_variation passed - Latent Variation: {diff:.4f}")

    def test_retraining_effect(self):
        latent_before = self.encoder.predict(self.image, verbose=0)
        self.autoencoder, self.encoder = dynamic_retraining_with_chaos_maps(self.autoencoder, self.encoder)
        latent_after = self.encoder.predict(self.image, verbose=0)
        diff = np.mean(np.abs(latent_before - latent_after))
        mse = float(self.autoencoder.evaluate(self.image, self.image, verbose=0))
        self.assertGreater(diff, 0.001, "Retrain effect too low")
        print(f"Test retraining_effect passed - Retrain Effect: {diff:.4f}, MSE after retraining: {mse:.4f}")

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

E.

Test latent_variation passed - Latent Variation: 1.0684


F
ERROR: test_encryption_decryption (__main__.TestImageBasedCrypto)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_1108\1674249446.py", line 19, in test_encryption_decryption
    ciphertext = public_key.encrypt(
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\cryptography\hazmat\backends\openssl\rsa.py", line 550, in encrypt
    return _enc_dec_rsa(self._backend, self, plaintext, padding)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\cryptography\hazmat\backends\openssl\rsa.py", line 97, in _enc_dec_rsa
    return _enc_dec_rsa_pkey_ctx(backend, key, data, padding_enum, padding)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\cryptography\hazmat\backends\openssl\rsa.py", line 163, in _enc_dec_rsa_pkey_ctx
    raise ValueError("Encryption/decryption failed.")
ValueError: Encrypti

## Итоги

Новая архитектура и многопоточность улучшили производительность (~0.838 сек), но ключ 1024 бита устарел. Финальная версия увеличит ключ до 2048 бит.